In [1]:
import os
import json
import random

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent
from react_agent.src.agents.triage import Triage

from react_agent.src.util.tools_fabric import ToolsFabric

from experiments.models.experiment_models import LabeledQAPairFacts
from experiments.util.fact_score.fact_scorer import FactScorer

In [2]:
USE_MCP = False
DEBUG_MODE = True

fact_scorer = FactScorer()
path_to_ressources = "./resources/01_facts/"

files = {
    "extensibility": "extensibility_assistance_facts.json",
    "malaysia": "malaysia_support_facts.json",
    "peppol": "peppol_support_facts.json",
    "all": "all_cases_facts.json",
}

In [3]:
file_path = os.path.abspath(path_to_ressources + files["all"])


with open(file_path, encoding="utf8") as f:
    data = json.load(f)
    qa_data_set = [LabeledQAPairFacts(**item) for item in data]

In [4]:
# Select 3 random elements
random_qa_pairs = random.sample(qa_data_set, 3)

In [5]:
triage_agent = Triage()

for qa_pair in random_qa_pairs:
    print(f"Asking agent question with id: {qa_pair.id}")
    query_categorization = triage_agent.triage_user_message(
        user_message=qa_pair.question
    )
    if DEBUG_MODE:
        print(f"Query categorization: {query_categorization}")

    tools = ToolsFabric.get_tools_for_category(
        use_mcp=USE_MCP,
        configuration=query_categorization["category"],
    )

    if USE_MCP:
        async with MultiServerMCPClient(tools) as client:
            agent = ReActAgent(tool_list=client.get_tools())
            await agent.arun_agent_with_input(
                user_message=query_categorization["user_query"], debug=DEBUG_MODE
            )
    else:
        agent = ReActAgent(tool_list=tools)
        agent.run_agent_with_input(
            user_message=query_categorization["user_query"], debug=DEBUG_MODE
        )

    run_data = agent.get_execution_data()

    run_data.pretty_print()

    print("Finished agent execution")

    print("Started FactScore calculation")
    output = await fact_scorer.get_fact_score(
        facts=qa_pair.facts,
        knowledge_source=run_data.final_output,
        debug=DEBUG_MODE,
    )
    print(f"Fact score: {output}")
    print("=" * 40)

Asking agent question with id: EA-3
Query categorization: {'user_query': 'For a Greece customer invoice process in Public Cloud, can I enhance or extend the generated XML before it is sent to the authorities?', 'category': 'Knowledge-QA'}
================================ Human Message =================================

For a Greece customer invoice process in Public Cloud, can I enhance or extend the generated XML before it is sent to the authorities?
================================== Ai Message ==================================

Initial Observation: The user is asking whether it is possible to enhance or extend the XML generated for a Greece customer invoice process in SAP Public Cloud before it is sent to the authorities.

Thought: To address this question, I need to understand the capabilities and limitations of SAP Document and Reporting Compliance in the context of XML generation and modification for eInvoicing, specifically for Greece. This involves checking if there are any cu